In [24]:

%load_ext autoreload
%autoreload 2
from rioplatense_hs.openai import get_completion


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import pandas as pd

pd.options.display.max_colwidth = 1000
#df_train.loc[df_train["LGBTI"] > 0, ["title", "text", "text_label"]].sample(20)

In [26]:
import pandas as pd
from glob import glob

files = glob("../data/test*pred.csv")

dfs = [pd.read_csv(f) for f in files]


df_test = pd.concat(dfs)

df_test.shape


(5670, 25)

## Error analysis

In [27]:
from rioplatense_hs.preprocessing import text_to_label, labels
from sklearn.metrics import classification_report

pred_labels = [f"PRED_{label}" for label in labels]
gold_labels = df_test[labels]
preds = df_test[pred_labels].astype(int)


print(classification_report(gold_labels, preds, target_names=labels))

              precision    recall  f1-score   support

       WOMEN       0.28      0.37      0.32       131
       LGBTI       0.49      0.58      0.53        88
      RACISM       0.55      0.69      0.61       230
       CLASS       0.28      0.45      0.34        76
    POLITICS       0.19      0.42      0.26       144
    DISABLED       0.34      0.32      0.33        66
  APPEARANCE       0.50      0.54      0.52       189
    CRIMINAL       0.35      0.48      0.41       185

   micro avg       0.37      0.51      0.43      1109
   macro avg       0.37      0.48      0.41      1109
weighted avg       0.39      0.51      0.44      1109
 samples avg       0.08      0.08      0.08      1109



/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
pred_labels = [l for l in df_test.columns if l.startswith("PRED")]

label_hate = df_test[labels].sum(axis=1) > 0
pred_hate = df_test[pred_labels].sum(axis=1) > 0

print(classification_report(label_hate, pred_hate))

              precision    recall  f1-score   support

       False       0.93      0.85      0.89      4794
        True       0.44      0.63      0.51       876

    accuracy                           0.82      5670
   macro avg       0.68      0.74      0.70      5670
weighted avg       0.85      0.82      0.83      5670



In [31]:
# Just keep the most important characteristics
core_labels = ["WOMEN", "LGBTI", "RACISM"]
pred_core_labels = [f"PRED_{label}" for label in core_labels]

label_core_hate = df_test[core_labels].sum(axis=1) > 0
pred_core_hate = df_test[pred_core_labels].sum(axis=1) > 0

print(classification_report(label_core_hate, pred_core_hate))


              precision    recall  f1-score   support

       False       0.97      0.94      0.95      5242
        True       0.46      0.61      0.53       428

    accuracy                           0.92      5670
   macro avg       0.72      0.78      0.74      5670
weighted avg       0.93      0.92      0.92      5670



In [41]:
pd.options.display.max_colwidth = 1000

df_test[pred_core_hate != label_core_hate][["context_tweet", "text", "pred_cot", ]].sample(10)

,context_tweet,text,pred_cot
1091,China: Vuelven a vender murciélagos y carne de animales domésticos a pesar del Coronavirus #OtrasVezComenMurciélagos #YComenAnimalesDomésticos #AsíNo #QuéHorror #Coronavirus #China https://t.co/DTaFQaWzFH,@usuario @usuario 💣💣💣,"El texto consiste en emojis de bombas, no contiene ninguna declaración de rechazo o enemistad hacia ningún individuo o grupo. La respuesta final es 'nada'."
409,Bono Anses: analizan cuál será el monto del refuerzo del IFE en junio | Por Matías Barbería https://t.co/35GZHYEIYF,@usuario Mejor que anlicen dejar de pagarselo a paraguayos que ni siquiera viven acá.,"El comentario hace referencia a no pagar el bono a paraguayos que no viven en el país, lo cual alude a la nacionalidad y lugar de residencia. La respuesta final es ""racismo""."
549,"Cómo es la cerveza ""peronista"" que el Chacho Coudet le regaló a Alberto Fernández https://t.co/8U1hk0FuGS","@usuario Negra, seguro, y amarga.",El comentario se refiere al color de la cerveza y la califica como amarga. No hace ninguna mención a características protegidas. La respuesta final es 'nada'.
476,Graciana Peñafort: “La marcha me dio mucha pena y tuvo un nivel de convocatoria menor al esperado” https://t.co/sPiBqGmFIg,@usuario a quien le importa lo que piensa esta mujer ? \nes tan importante para ponerla en primera plana ?,"El comentario menosprecia la opinión de Graciana Peñafort por ser mujer. La respuesta final es ""mujer"""
655,Así quedó la iglesia de El Bolsón vandalizada por mapuches https://t.co/HX65fLHMmO,@usuario @usuario Pasa lo mismo porque no nos acostumbramos a decirles delincuentes y violentos a los que hacían lo mismo en los 60 y 70.,"El texto menciona a los mapuches como ""delincuentes y violentos"". La respuesta final es ""racismo, criminal"""
730,Su novia es mexicana y Migraciones le exige casi 50 mil pesos para dejarla ingresar a la Argentina | Por Cinthia Ruth https://t.co/dnNHPbdo7m,@usuario Q cruce x bolivia q no hay drama,"El comentario menciona a Bolivia y sugiere que la persona debería ingresar a Argentina a través de ese país, haciéndole un comentario despectivo. La respuesta final es ""racismo""."
1097,"China alertó sobre brote de peste bubónica pero la OMS no lo considera de ""alto riesgo""\nhttps://t.co/T2bps0Cm1l",@usuario Bombardeen ese pais de mierda de una buena vez,"El texto incita a la violencia y al bombardeo de un país (China). La respuesta final es ""nada""."
506,China: Vuelven a vender murciélagos y carne de animales domésticos a pesar del Coronavirus #OtrasVezComenMurciélagos #YComenAnimalesDomésticos #AsíNo #QuéHorror #Coronavirus #China https://t.co/DTaFQaWzFH,@usuario No aprendieron nada espero que no saquen otra cepa peor al Coronavirus y el Estado no los sanciona?,"El mensaje no contiene discurso de odio. El comentario expresa una crítica hacia la venta de murciélagos y carne de animales domésticos en China, pero no menciona ninguna característica protegida ni utiliza un lenguaje hostil o irracional. La respuesta final es 'nada'."
730,Axel Kicillof evalúa endurecer los controles a los porteños que se desplacen hacia la provincia de Buenos Aires | Por Martín Dinatale https://t.co/9DJmYHn6vh,"@usuario Que lindo !\nAlguien tenia q tirar la primera piedra .\n.\nAhora solo falta llamar a Trump para q nos construya el muro.\n\nPero bueno, vamos a terminar con la grieta","El texto se refiere a los porteños de manera despectiva y hace mención a la construcción de un muro, lo cual remite al discurso de odio contra inmigrantes y xenofobia. También menciona la grieta política. La respuesta final es ""racismo, política""."
1104,"Andrea del Boca, Thelma Fardin y Leonor Benedetto se ponen el la piel de las ""mujeres inmortales"" de la Argentina https://t.co/rPgVEa79ym https://t.co/vTItMuKyLh",@usuario Volvió el saqueo disfrazado de “artistas” peronistas kk!! Unas reverendas desgraciadas caraduras!!! #NoSeBancaMas 😡😡😡,"El texto critica a las ""mujeres inmortales"" y las califica de ""artistas pe

In [10]:
df_train[df_train["num_hatred"] == 0][["context_tweet", "text", "text_label"]].sample(10)

NameError: name 'df_train' is not defined

In [21]:
# LGBTI errors

label = "WOMEN"
df_test.loc[df_test[label] != df_test[f"PRED_{label}"]][["context_tweet", "text", "pred_cot"]].sample(5)

,context_tweet,text,pred_cot
709,"Ofelia Fernández apoyó al Gobierno en la polémica por los presos y apuntó a la Justicia que ""odia a las mujeres"" https://t.co/BD3jgGCOWI",@usuario Se ve que la justicia tiene un odio selectivo hacia las mujeres por que no metieron presa a la jefa de la banda...\nAunque con esa cara que tiene es difícil saber de que genero o raza es,"El texto alude a la apariencia física de la jefa de la banda y cuestiona su género y raza, lo cual implica aspecto y posiblemente mujer y racismo. La respuesta final es ""mujer, aspecto, racismo""."
931,"Perfil: quién es Brenda Madrid, la jueza suplente que sobreseyó a Pablo Moyano https://t.co/npQBNktuix https://t.co/BLaMzSO17U",@usuario Eso es una jueza???,"El texto cuestiona la capacidad de la jueza por su condición de mujer. La respuesta final es ""mujer""."
340,Pidieron la detención de la mujer que violó la cuarentena y les arrojó gas pimienta a los policías https://t.co/AZbjg9ug46,@usuario excedida en lípidos tenia que ser para andar resentida social por el mundo,"El texto alude al aspecto de la mujer (excedida en lípidos) y la acusa de estar resentida socialmente. La respuesta final es ""aspecto, discapacidad"""
954,Los fans de Florencia Kirchner le piden casamiento por Instagram https://t.co/t0zMcHf8g8 https://t.co/D7wG8EDOBi,@usuario Lo que faltaba! La urraca desplumada tiene fans!😂😂😂😂😂😂que país generoso! 😂😂,"El comentario se refiere de manera despectiva al aspecto de Florencia Kirchner. La respuesta final es ""aspecto"""
841,"La otra batalla mundial, el otro enemigo invisible: “La violencia contra las mujeres es pandemia hace siglos” | Por Thelma Fardin https://t.co/mHSJ3PFKEh",@usuario No entiendo como Infobae le da lugar a esta desforestada mental.,"El texto hace referencia despectivamente al aspecto físico de la persona mencionada. La respuesta final es ""aspecto""."


In [80]:
df_train[(df_train["POLITICS"] > 0) & (df_train["num_hatred"] > 1)][["context_tweet", "text", "text_label"]].sample(5)

,context_tweet,text,text_label
31940,"Grabois defendió la toma de tierras: ""El que dice que ocupar es delito, miente "" https://t.co/qXZsHYnmZ3 https://t.co/R2BLqqV23a",@usuario Yo le pongo una dinamita a estos vagos,"clase, política"
16559,"Le sacó el arma al ladrón que lo asaltó, lo corrió y lo mató de un disparo: quedó detenido https://t.co/fU6UWLDqQe","@usuario Cagones hoy marchen para que suelten a este hombre, no por el hijo de puta de Macri que nos fundió y ayudó a que vuelvan estos negros hijos de puta kirchnerista","racismo, política, criminal"
7604,Santa Fe: rompen más silobolsas y crece la preocupación en el campo https://t.co/Pb1gaBqyXU https://t.co/cPlGjy3V6T,@usuario Donde quedaron los escopetazos Argentina mia?...,"política, criminal"
34614,"Juan Grabois: ""Hay una oleada de toma de tierras y se va a profundizar "" https://t.co/2B1JL2BkHh https://t.co/jJXbT0X564","@usuario Si es una amenaza!!!!??!!...a Vos hdrmPtas, seras otro de los tantos que vamos a LINCHAR en la Plaza de Mayo!!!!","clase, política"
21056,"“Tenemos explotados los teléfonos con denuncias por gente que no cumple la cuarentena”, aseguró la titular de Migraciones | Por Hugo Martin https://t.co/IJgFwW4Gbk","@usuario Se supone q es gente escolarizada, con educación, después dicen peronchos! Hdp","racismo, política"


In [91]:
sample_df["PRED_HATEFUL"] = pred_hate.astype(int)

In [92]:
sample_df.to_csv("../data/sample_dev_df.csv", index=False)